In [2]:
import os
import json
import geopandas as gpd
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql.functions import to_date, col, lit, substring, regexp_replace
from pyspark.sql.types import DoubleType, StructType, StructField, StringType

# ==============================================================================
# ÉTAPE 1 : PRÉPARATION DES DONNÉES (ETL SPARK)
# ==============================================================================
print("🔄 Démarrage de l'ETL Spark...")

# 1. Initialisation Spark
try:
    spark
except NameError:
    spark = SparkSession.builder.appName("OneCode_ETL").getOrCreate()

# 2. Configuration des chemins
GEOJSON_URL = "https://raw.githubusercontent.com/nycehs/NYC_geography/master/UHF42.geo.json"
AIR_QUALITY_PATH = "/home/jovyan/work/data/air_quality/nyc_air_quality_raw.json" 
WEATHER_PATH = "hdfs://namenode:9000/user/mathis/datalake/noaa_gsod_nyc_raw_2005_2023.parquet"

# --- A. Traitement Géographique (GeoJSON) ---
print("   📍 Traitement du GeoJSON...")
gdf_quartiers = gpd.read_file(GEOJSON_URL)

# Correction Géométrique : On projette en EPSG:2263 (NYC Feet) pour calculer le centre, puis on revient en Lat/Lon
# Cela évite le warning et donne un centre plus précis.
gdf_quartiers = gdf_quartiers.to_crs(epsg=2263) 
gdf_quartiers['centroid'] = gdf_quartiers.geometry.centroid
gdf_quartiers = gdf_quartiers.to_crs(epsg=4326) # Retour au standard GPS

# Extraction Lat/Lon des centroïdes recalculés
# Attention: Après reprojection, on accède au centroid via la colonne qu'on a créée, mais il faut la reprojeter aussi
# Plus simple : on recrée le centroid en 4326 directement si la précision au mètre n'est pas vitale, 
# mais pour être propre, utilisons la colonne geometry reprojetée.
gdf_quartiers['LATITUDE_ZONE'] = gdf_quartiers['centroid'].to_crs(epsg=4326).y
gdf_quartiers['LONGITUDE_ZONE'] = gdf_quartiers['centroid'].to_crs(epsg=4326).x

# Sauvegardes
gdf_quartiers[['GEOCODE', 'GEONAME', 'BOROUGH', 'geometry']].to_file("dashboard_map.geojson", driver='GeoJSON')
pdf_locations = pd.DataFrame(gdf_quartiers[['GEOCODE', 'GEONAME', 'LATITUDE_ZONE', 'LONGITUDE_ZONE']])
spark_locations = spark.createDataFrame(pdf_locations)

# --- B. Traitement Air Quality (CORRIGÉ) ---
print("   💨 Traitement Air Quality (Mode Manuel)...")

# Lecture manuelle car le JSON est une liste de listes (Socrata)
with open(AIR_QUALITY_PATH, 'r') as f:
    raw_data = json.load(f)

# Définition du Schéma explicite (Index Socrata)
# 17=Date, 14=GeoID, 10=Polluant, 18=Valeur
fields = []
for i in range(20): # On crée 20 colonnes génériques
    name = f"col_{i}"
    if i == 17: name = "DATE_MESURE_BRUTE"
    elif i == 14: name = "GEOJOIN_ID_BRUT"
    elif i == 10: name = "NOM_POLLUANT"
    elif i == 18: name = "VALEUR_MESURE_BRUTE"
    fields.append(StructField(name, StringType(), True))

schema = StructType(fields)

# Création DataFrame Spark
air_q_df = spark.createDataFrame(raw_data, schema=schema)

# Nettoyage
air_quality_clean = air_q_df.withColumn(
    "DATE_OBSERVATION", 
    to_date(col("DATE_MESURE_BRUTE"))
).select(
    col("GEOJOIN_ID_BRUT").alias("GEOJOIN_ID"),
    col("DATE_OBSERVATION"),
    col("NOM_POLLUANT"),
    col("VALEUR_MESURE_BRUTE").cast(DoubleType()).alias("VALEUR")
).filter(col("VALEUR").isNotNull())

# --- C. Jointure Air Quality + Coordonnées ---
print("   🔗 Jointure Air Quality + Géographie...")
final_air_data = air_quality_clean.join(
    spark_locations,
    air_quality_clean.GEOJOIN_ID == spark_locations.GEOCODE,
    "inner"
).drop("GEOCODE")

final_air_data.toPandas().to_parquet("dashboard_data_air.parquet", index=False)

# --- D. Traitement Météo ---
print("   ☀️  Traitement Météo...")
weather_df = spark.read.parquet(WEATHER_PATH)
weather_lite = weather_df.select("ID_STATION", "NAME", "LATITUDE", "LONGITUDE", "DATE", "TEMP", "DEWP", "WDSP")
weather_lite.toPandas().to_parquet("dashboard_data_weather.parquet", index=False)

print("✅ ETL terminé ! Fichiers générés.")

# ==============================================================================
# ÉTAPE 2 : GÉNÉRATION DU DASHBOARD (Identique)
# ==============================================================================
dashboard_code = """
import streamlit as st
import pandas as pd
import geopandas as gpd
import folium
from streamlit_folium import st_folium
import plotly.express as px
import numpy as np
from math import radians, cos, sin, asin, sqrt

st.set_page_config(layout="wide", page_title="NYC Environmental Dashboard")

@st.cache_data
def load_data():
    geo = gpd.read_file("dashboard_map.geojson")
    air = pd.read_parquet("dashboard_data_air.parquet")
    weather = pd.read_parquet("dashboard_data_weather.parquet")
    
    air['DATE_OBSERVATION'] = pd.to_datetime(air['DATE_OBSERVATION'])
    weather['DATE'] = pd.to_datetime(weather['DATE'])
    
    stations = weather[['ID_STATION', 'NAME', 'LATITUDE', 'LONGITUDE']].drop_duplicates()
    return geo, air, weather, stations

def haversine_dist(lon1, lat1, lon2, lat2):
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])
    a = sin((lat2-lat1)/2)**2 + cos(lat1) * cos(lat2) * sin((lon2-lon1)/2)**2
    return 6371 * 2 * asin(sqrt(a))

def get_weighted_weather(target_lat, target_lon, weather_df, station_ids):
    subset = weather_df[weather_df['ID_STATION'].isin(station_ids)].copy()
    if subset.empty: return None
    subset['dist'] = subset.apply(lambda x: haversine_dist(target_lon, target_lat, x['LONGITUDE'], x['LATITUDE']), axis=1)
    subset['weight'] = 1 / (subset['dist'] + 0.1)
    try:
        return np.average(subset['TEMP'], weights=subset['weight'])
    except:
        return None

geo, df_air, df_weather, df_stations = load_data()

st.sidebar.title("Filtres")
dates = st.sidebar.date_input("Période", [df_air['DATE_OBSERVATION'].min(), df_air['DATE_OBSERVATION'].max()])
polluant = st.sidebar.selectbox("Polluant", df_air['NOM_POLLUANT'].unique())

mask_air = (df_air['DATE_OBSERVATION'].dt.date >= dates[0]) & (df_air['DATE_OBSERVATION'].dt.date <= dates[1])
df_air_view = df_air[mask_air & (df_air['NOM_POLLUANT'] == polluant)]

col_map, col_stats = st.columns([3, 2])

with col_map:
    st.subheader("Carte des Quartiers")
    m = folium.Map([40.7, -74.0], zoom_start=10)
    folium.GeoJson(
        geo, 
        name="Quartiers",
        tooltip=folium.GeoJsonTooltip(fields=['GEONAME', 'BOROUGH']),
        style_function=lambda x: {'fillColor': '#3388ff', 'color': 'black', 'weight': 0.5, 'fillOpacity': 0.4},
        highlight_function=lambda x: {'weight': 3, 'color': 'red'}
    ).add_to(m)
    st_map = st_folium(m, width=None, height=600)

selected_geocode = None
selected_name = st.sidebar.selectbox("Ou choisir un quartier :", geo['GEONAME'].unique())
q_data = geo[geo['GEONAME'] == selected_name].iloc[0]
selected_geocode = q_data['GEOCODE']
centroid = q_data.geometry.centroid
lat_q, lon_q = centroid.y, centroid.x

with col_stats:
    st.title(selected_name)
    st.info(f"Borough: {q_data['BOROUGH']}")
    
    st.markdown("### 1. Qualité de l'Air")
    local_air = df_air_view[df_air_view['GEOJOIN_ID'] == selected_geocode]
    if not local_air.empty:
        st.metric(f"Moyenne {polluant}", f"{local_air['VALEUR'].mean():.2f}")
        fig = px.bar(local_air, x='DATE_OBSERVATION', y='VALEUR', title="Évolution")
        st.plotly_chart(fig, use_container_width=True)
    else:
        st.warning("Pas de données.")

    st.markdown("### 2. Météo Pondérée")
    df_stations['dist'] = df_stations.apply(lambda x: haversine_dist(lon_q, lat_q, x['LONGITUDE'], x['LATITUDE']), axis=1)
    df_stations = df_stations.sort_values('dist')
    sel_stations = st.multiselect("Stations", df_stations['ID_STATION'].tolist(), format_func=lambda x: f"{df_stations[df_stations.ID_STATION==x].iloc[0]['NAME']} ({df_stations[df_stations.ID_STATION==x].iloc[0]['dist']:.1f}km)")
    
    if sel_stations:
        mask_w = (df_weather['DATE'].dt.date >= dates[0]) & (df_weather['DATE'].dt.date <= dates[1])
        w_temp = get_weighted_weather(lat_q, lon_q, df_weather[mask_w], sel_stations)
        if w_temp: st.metric("Température Pondérée", f"{w_temp:.1f} °F")
"""

with open("app.py", "w", encoding='utf-8') as f:
    f.write(dashboard_code)

print("\n🚀 Application générée ! Lancez dans le terminal : streamlit run app.py")

🔄 Démarrage de l'ETL Spark...
   📍 Traitement du GeoJSON...
   💨 Traitement Air Quality (Mode Manuel)...
   🔗 Jointure Air Quality + Géographie...
   ☀️  Traitement Météo...
✅ ETL terminé ! Fichiers générés.

🚀 Application générée ! Lancez dans le terminal : streamlit run app.py
